### Notebook to adjust PetroNER for Entity Linking task

In [10]:
from owlready2 import *
import gensim
from conllu import parse_incr, parse

Load ontology and embeddings

In [4]:
#OntoGeoLógica populada (OntoGeoLogicaPopulada.owl)
onto = get_ontology("../../KnowledgeGraph/OntoGeoLogicaPovoadaInstanciasRelacoes.owl")
onto.load()

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#has_beginning

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#has_end

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#interval_contains

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#interval_during

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#interval_in

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#interval_finished_by

* Owl

get_ontology("http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#")

In [ ]:
# Modelo OWL2Vec - 
model_owl2v = gensim.models.Word2Vec.load("../../Embeddings/PetroOntoVec/PetroOntoVec_simples/outputontology.embeddings")

Load PetroNER (file in CONLLU format)

In [11]:
# CONLLU parser
boleltins = open("../PetroNER/petroner-2022-12-19.conllu", "r", encoding="utf-8")
sentences = parse_incr(boleltins)

### Preparing and processing the dataset

Função que recebe o texto anotado em formato CONLLU e processa para o treinamento. Tem como saída as sentenças acrescida das tags para indicar as entidades, bem como com a entidade no início do texto; lista com as entidades (instâncias extraídas do texto); e lista com as classes que as entidades pertencem. O texto sai com o seguinte formato:  
"[Nome da classe] Nome da entidade | blablabla blablabla blablabla [E] Nome da entidade [/E] blablabla blablabla." 

In [13]:
# Função que recebe
def preparar_dataset(tokenlist):

    #sent = TokenList([])
    sent_orig = ''   # String para compor a sentença original
    sents_NER = []   # Lista para armazenar as sentenças com indicação das entidades
    NER_on = False   # Variável para indicar se está iterando por token de entidades
    NER_tok = ''     # string para armazenar os tokens das entidades que estão sendo iteradas
    MW_on = False    # Variável para indicar se está iterando por tolkens multiword
    MW_tok = ''      # string para armazenar o termo multiword
    MW_end = 0       # ID final do token multiword
    ENT = []
    ENT_class = []

    for tok in tokenlist:
        # ignorando id não inteiro (tuplas), ou seja token multiword
        if type(tok['id']) == int:

            # Verifica se o token inicia uma entidade ("B")
            if tok['deps'][0] == 'B':
                
                # Verifica se a variável NER_on está ativado (No caso de um tok "B" seguido de outro)
                if NER_on == True:
                    
                    NER_on = False  # Desativa a variável NER_on

                    # Para sentença mais recente da lista sents_NER, inclui a entidade no início e o marcador [/E] no final 
                    sents_NER[-1] = '[' + ENT_class[-1] + '] ' + NER_tok + '| ' + sents_NER[-1].strip() + ' [/E]'

                    # Incluir o NER_tok na lista ENT
                    ENT.append(NER_tok.strip())

                    NER_tok = '' # Apaga a string NER_tok

                NER_on = True  # Ativa a variável NER_on
                # Acrescenta uma sentença com o marcador [E] na listas de sentenças com NER
                sents_NER.append(sent_orig + ' [E]')
                # Incluir classe na lista ENT_class
                ENT_class.append(tok['deps'][2:])

            # Identifica o primeiro token "O" após uma sequência de entidades
            if (tok['deps'][0] == 'O') & NER_on:
                
                NER_on = False  # Desativa a variável NER_on

                # Para sentença mais recente da lista sents_NER, 
                # inclui a classe e a entidade no início e o marcador [/E] no final 
                sents_NER[-1] = '[' + ENT_class[-1] + '] ' + NER_tok + '| ' + sents_NER[-1].strip() + ' [/E]'

                # Incluir o NER_tok na lista ENT
                ENT.append(NER_tok.strip())

                NER_tok = '' # Apaga a string NER_tok
                
            # Identifica se o último token da sequência tem o marcador "B" ou "I".    
            if (tokenlist[-1]['id'] == tok['id']) & ((tok['deps'][0] == 'B') | (tok['deps'][0] == 'I')):
                NER_on = False  # Desativa a variável NER_on

                # Para sentença mais recente da lista sents_NER, 
                # inclui a classe e a entidade no início e o marcador [/E] no final 
                sents_NER[-1] = '[' + ENT_class[-1] + '] ' + NER_tok + '| ' + sents_NER[-1].strip() + ' [/E]'

                # Incluir o NER_tok na lista ENT
                ENT.append(NER_tok.strip())

                NER_tok = '' # Apaga a string NER_tok

            # Verifica se está iterando por uma multiword ou não
            if not MW_on:

                # Acrecenta o token à string da entidade
                if NER_on:
                    NER_tok = NER_tok + tok['form'] + ' '

                # Se o token for uma pontuação, não inclui espaço entre o token anterior 
                if tok['upos'] == 'PUNCT':
                    # O token é incluído na senteça original e em todas as sentenças da lista sents_NER
                    sent_orig = sent_orig + tok['form']
                    for i in range(len(sents_NER)):
                        sents_NER[i] = sents_NER[i] + tok['form']
                else:
                    sent_orig = sent_orig + ' ' + tok['form']
                    for i in range(len(sents_NER)):
                        sents_NER[i] = sents_NER[i] + ' ' + tok['form']

            else:
                # Se o token for o último termo da multiword, desativamos a variável MW_on
                if tok['id'] == MW_end:
                    MW_on = False

                    # Acrecenta a multiword à string da entidade
                    if NER_on:
                        NER_tok = NER_tok + MW_tok + ' '

                    #Ao invés de incluir nas senteças o token, vamos incluir o MW_tok 
                    # Se o token for uma pontuação, não inclui espaço entre o token anterior 
                    if tok['upos'] == 'PUNCT':
                        # O token é incluído na senteça original e em todas as sentenças da lista sents_NER
                        sent_orig = sent_orig + MW_tok
                        for i in range(len(sents_NER)):
                            sents_NER[i] = sents_NER[i] + MW_tok
                    else:
                        sent_orig = sent_orig + ' ' + MW_tok
                        for i in range(len(sents_NER)):
                            sents_NER[i] = sents_NER[i] + ' ' + MW_tok

        # Se a ID do token for referente a uma multiword, ativamos a variável MW_on, 
        # armazenamos o termo MW_tok e o valor da 'id' final
        else:
            MW_on = True
            MW_tok = tok['form']
            MW_end = tok['id'][-1]
    
    return(sents_NER, ENT, ENT_class)

In [19]:
# Separando as listas de sentenças, entidades e classes
text = []
entities = []
classes_NER = [] 

# Iterando por todas as sentenças em formato CONLLU para preparar o dataset
n = 0
for tokenlist in sentences:
    print(tokenlist)
    n = n + 1

    text_n, entities_n, classes_NER_n = preparar_dataset(tokenlist)
    
    text = text + text_n
    entities = entities + entities_n
    classes_NER = classes_NER + classes_NER_n
    if n > 1:
        break
    
print('Total de sentenças anotadas: ', n)
print('N° de sentenças: ', len(text))
print ('N° de entidades: ', len(entities))
print('N° de classes: ', len(classes_NER))

TokenList<O, Boletim, tem, servido, como, um, instrumento, de, divulgação, de, novas, idéias, e, conceitos, para, profissionais, que, atuam, em, mais, de, 600, instituições, nacionais, e, internacionais, ,, contribuindo, para, a, expansão, do, de, o, conhecimento, das, de, as, bacias, sedimentares, brasileiras, ., ., metadata={text: "O Boletim tem servido como um instrumento de divulgação de novas idéias e conceitos para profissionais que atuam em mais de 600 instituições nacionais e internacionais, contribuindo para a expansão do conhecimento das bacias sedimentares brasileiras..", sent_id: "boletins-000001-28"}>
TokenList<Com, a, nova, política, do, de, o, setor, petróleo, ,, a, expectativa, é, que, o, Boletim, passe, a, contar, com, a, contribuição, de, geocientistas, de, outras, companhias, que, venham, a, atuar, no, em, o, território, brasileiro, ., ., metadata={text: "Com a nova política do setor petróleo, a expectativa é que o Boletim passe a contar com a contribuição de geocien

In [20]:
text

['[BACIA] bacias | O Boletim tem servido como um instrumento de divulgação de novas idéias e conceitos para profissionais que atuam em mais de 600 instituições nacionais e internacionais, contribuindo para a expansão do conhecimento das [E] bacias [/E] sedimentares brasileiras..']